In [3]:
from neo4j import GraphDatabase
import requests
import time
import pprint
import json
import os

In [30]:
pp = pprint.PrettyPrinter(indent=4)

In [4]:
# DEFINE the following environment variables:

NEO4J_URI           = "neo4j://localhost:7687"
NEO4J_USER          = "neo4j"
NEO4J_PASSWORD      = "letmein"
NEO4J_DATABASE      = "landgraph"
PROPUBLICA_API_KEY  = os.environ.get("PROPUBLICA_API_KEY")
PROPUBLICA_ENDPOINT = 'https://api.propublica.org/congress/v1/bills/search.json?query="{name}"'

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [10]:
# TODO: download PP geojsons and load into Neo4j

In [41]:
# TODO: query Neo4j for parcel names

# Let's start with only federally owned parcels for now
PARCEL_NAME_QUERY = """
MATCH (p:Parcel) WHERE p.Own_Type = "FED"
RETURN p.name AS name
"""

IMPORT_BILL_QUERY = """
WITH $bill AS bill
MATCH (p:Parcel {name: $name})
MERGE (b:Bill {billId: bill.bill_id})
SET b.number          = bill.number,
    b.title           = bill.title,
    b.short_title     = bill.short_title,
    b.introduced_date = bill.introduced_date
MERGE (l:Legislator {legislatorId: bill.sponsor_id})
ON CREATE SET
    l.name = bill.sponsor_name,
    l.state = bill.sponsor_state,
    l.party = bill.sponsor_party
MERGE (c:Committee {name: bill.committees})
MERGE (b)<-[:SPONSORED]-(l)
MERGE (b)-[:LEGISLATES]->(p)
MERGE (b)-[:IN_COMMITTEE]->(c)
"""

In [44]:
def get_parcel_names(tx):
    parcels = []
    result = tx.run(PARCEL_NAME_QUERY)
    for record in result:
        parcels.append(record["name"])
    return parcels


def write_bills(tx, bill, parcel):
    result = tx.run(IMPORT_BILL_QUERY, bill=bill, name=parcel)
    return result

In [27]:

# Fetch all parcel names from Neo4j
with driver.session(database=NEO4J_DATABASE) as session:
    parcels = session.execute_read(get_parcel_names)
    print(parcels)


['Humboldt River Field Office', 'Humboldt National Forest', 'Hart Mountain National Antelope Refuge', 'Pinedale Field Office', 'Pocatello Field Office', 'Salt Lake  Field Office', 'Willow Bay', 'Anderson Ranch Reservoir', 'Blackfoot Reservoir', 'Little Wood Reservoir', 'Minidoka Project - American Falls Reservoir', 'Palisades Reservoir', 'Cache National Forest', 'Craters of the Moon National Monument', 'Bear Lake National Wildlife Refuge', 'Farm Service Agency Interest Of Id', 'Hagerman National Fish Hatchery', 'Magic Valley Hatchery', 'Indian Creek Reservoir Access Site', 'Ted Trueblood Wildlife Habitat Area', 'Deer Parks Wildlife Mitigation Unit', 'Chinks Peak Radio Repeater', 'Pit Tag Right Of Way', 'Snake River Vista Access Site', 'Twana Gulch Access Site', 'Castle Rocks State Park', 'Cartier Slough Wildlife Management Area', 'Lake Walcott State Park', 'Miller Park', 'Crane Prairie Reservoir', 'Phillips Reservoir', 'Prineville Reservoir', 'Unity Reservoir', 'Warm Springs Reservoir'

In [47]:
# TODO: Query Propublica API

headers = {'X-API-Key': PROPUBLICA_API_KEY}

for parcel in parcels:
    try:
        r = requests.get(PROPUBLICA_ENDPOINT.format(name=parcel), headers=headers)
        response = r.json()
        for result in response['results']:
            for bill in result['bills']:
                with driver.session(database=NEO4J_DATABASE) as session:
                    session.execute_write(write_bills, bill, parcel)
        print(parcel)
        time.sleep(3)
    except:
        pass

Humboldt River Field Office
Humboldt National Forest
Hart Mountain National Antelope Refuge
Pinedale Field Office
Pocatello Field Office
Salt Lake  Field Office
Willow Bay
Anderson Ranch Reservoir
Blackfoot Reservoir
Little Wood Reservoir
Minidoka Project - American Falls Reservoir
Palisades Reservoir
Cache National Forest
Craters of the Moon National Monument
Bear Lake National Wildlife Refuge
Farm Service Agency Interest Of Id
Hagerman National Fish Hatchery
Magic Valley Hatchery
Indian Creek Reservoir Access Site
Ted Trueblood Wildlife Habitat Area
Deer Parks Wildlife Mitigation Unit
Chinks Peak Radio Repeater
Pit Tag Right Of Way
Snake River Vista Access Site
Twana Gulch Access Site
Castle Rocks State Park
Cartier Slough Wildlife Management Area
Lake Walcott State Park
Miller Park
Crane Prairie Reservoir
Phillips Reservoir
Prineville Reservoir
Unity Reservoir
Warm Springs Reservoir
Wickiup Reservoir
Malheur National Forest
Umatilla National Forest
Ochoco Reservoir Public Fishing Ar